### Import libraries

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import codecs
import collections
import nltk
import pickle

### Import companieslist

In [2]:
# Récupére la liste des compagnies privées du domaine spatial
with open('List_private_spaceflight_company.pickle','rb') as fp:
    companieslist = pickle.load(fp)

# Ce dictionnaire est utilisé plus tard pour définir les sources, targets et id
complist = {}
i = 0
for company in companieslist:
    i += 1
    complist[company] = i

# complist

### Initialization of localisation liste

In [3]:
localisation = ['Mercury', 'Venus', 'Moon', 'Mars',
                'Jupiter', 'Saturn', 'Uranus', 'Neptune']

# Ce dictionnaire est utilisé plus tard pour définir les sources, targets et id
loclist = {}
for loc in localisation:
    i += 1
    loclist[loc] = i

# loclist

### Initialization of the nodes list use in Gephi software

In [4]:
gephinodelist = {'Id': [], 'Label': [], 'Type': []}
#                  'Start': [], 'End': []}

for key in complist.keys():
    gephinodelist['Label'].append(key)
    gephinodelist['Id'].append(complist[key])
    
for key in loclist.keys():
    gephinodelist['Label'].append(key)
    gephinodelist['Id'].append(loclist[key])
    
for i in range(0,len(gephinodelist['Label'])):
    if gephinodelist['Label'][i] in companieslist:
        gephinodelist['Type'].append('company')
    else:
        gephinodelist['Type'].append('localisation')
#     gephinodelist['Start'].append(2010)
#     gephinodelist['End'].append(2019)
        
df_1 = pd.DataFrame(data=gephinodelist)
df_1

,Id,Label,Type
0,1,ARCA Space Corporation,company
1,2,Australian Space Research Institute,company
2,3,Ventions,company
3,4,Blue Origin,company
4,5,Canadian Arrow,company
5,6,Exos Aerospace,company
6,7,Firefly Aerospace,company
7,8,Gilmour Space Technologies,company
8,9,Generation Orbit Launch Services,company
9,10,Independence-X Aerospace,company


In [6]:
df_1.to_excel('NodesList_Gephi.xlsx')

### Fonctions

In [74]:
# Analyse des différents HTML
def analyse_html(html, companieslist):
    soup = BeautifulSoup(html)
    
    articles = soup.find_all('section', class_='content-wrapper')
    item = ''
    for article in articles:
        texts = article.find_all('p')
        for text in texts:
            newitem = text.get_text()
            item = item + newitem + '\n\n'

    comps = set([company for company in companieslist if company in item])

    tockens = nltk.word_tokenize(item)
    locs = set([tocken for tocken in tockens if tocken in localisation])
    
    return comps, locs

In [79]:
gephiedgeslist = {'Source': [], 'Target': [], 'ID': [], 'Type': [], 'Interval': [], 'Weight': []}

id_relationlist = {}
# Affecte une identitée (integer) à chaque relation
# {'13-30': 1, '13-29': 2, '22-29': 3, '22-30': 4
# La relation 13-30 à pour Id 1

ite = 0
weight = {} 
for i in range(0,9):
    print('201' + str(i))

    for file in os.listdir('US02_Python_Project_HTML/Ar_201' + str(i) + '/'):
        if file.endswith('.html'):
            with open('US02_Python_Project_HTML/Ar_201' + str(i) + '/' + file, encoding = 'utf-8') as fp:

                html = fp.read()
                comps, locs = analyse_html(html, companieslist)

                if len(comps) > 0 and len(locs) > 0:
                    indice_complist = [complist[comp] for comp in comps]
                    indice_loclist = [loclist[loc] for loc in locs]
                    
                    for comp in indice_complist:
                        for loc in indice_loclist:
                            relation = str(comp) + '-' + str(loc) # Exemple: 13-30
                            if relation not in id_relationlist.keys():
                                
                                ite += 1
                                id_relationlist[relation] = ite
                                
                                gephiedgeslist['Source'].append(comp)
                                gephiedgeslist['Target'].append(loc)
                                gephiedgeslist['ID'].append(id_relationlist[relation])
                                gephiedgeslist['Type'].append('Directed')
                                gephiedgeslist['Interval'].append('[2010,2019]')
                                gephiedgeslist['Weight'].append(0)
                                weight[ite] = 1
                            else:
                                weight[id_relationlist[relation]] += 1
                                
    for id_re in weight.keys():
        we = weight[id_re]
        if i == 0 or gephiedgeslist['Weight'][id_re-1] == 0:
#             gephiedgeslist['Interval'][id_re-1] = ('[201' + str(i))
            gephiedgeslist['Weight'][id_re-1] = ('[201' + str(i) + ', 201' + str(i+1) + ', ' + str(we) + ']')
        else:
#             gephiedgeslist['Interval'][id_re-1] += ', 201' + str(i)
            gephiedgeslist['Weight'][id_re-1] += '; [201' + str(i) + ', 201' + str(i+1) + ', ' + str(we) + ']'
        weight[id_re] = 0
            
# for j in range(0,len(gephiedgeslist['Interval'])):
#     gephiedgeslist['Interval'][j] += ']'

2010
2011
2012
2013
2014
2015
2016
2017
2018


In [80]:
df_2 = pd.DataFrame(data=gephiedgeslist)
df_2

,Source,Target,ID,Type,Interval,Weight
0,13,30,1,Directed,"[2011,2018]","[2010, 2011, 21]; [2011, 2012, 24]; [2012, 201..."
1,13,29,2,Directed,"[2011,2018]","[2010, 2011, 3]; [2011, 2012, 4]; [2012, 2013,..."
2,22,29,3,Directed,"[2011,2018]","[2010, 2011, 7]; [2011, 2012, 5]; [2012, 2013,..."
3,22,30,4,Directed,"[2011,2018]","[2010, 2011, 20]; [2011, 2012, 42]; [2012, 201..."
4,22,32,5,Directed,"[2011,2018]","[2010, 2011, 2]; [2011, 2012, 7]; [2012, 2013,..."
5,23,30,6,Directed,"[2011,2018]","[2010, 2011, 5]; [2011, 2012, 14]; [2012, 2013..."
6,11,34,7,Directed,"[2011,2018]","[2010, 2011, 2]; [2011, 2012, 0]; [2012, 2013,..."
7,22,33,8,Directed,"[2011,2018]","[2010, 2011, 2]; [2011, 2012, 1]; [2012, 2013,..."
8,25,33,9,Directed,"[2011,2018]","[2010, 2011, 1]; [2011, 2012, 1]; [2012, 2013,..."
9,25,30,10,Directed,"[2011,2018]","[2010, 2011, 4]; [2011, 2012, 10]; [2012, 2013..."


In [81]:
df_2.to_excel('EdgesList_Gephi.xlsx')